In [1]:
from toolz import valmap

import h5py
import pyharm

import matplotlib.pyplot as plt
import numpy as np
from numpy import ma
from matplotlib import ticker, cm
import pyharm.plots.plot_dumps as pplt
from scipy import signal

from mpl_toolkits import mplot3d

In [2]:
i0 = 5010 # initial  frame
i1 = 5061 # 1 + last frame

fmt = 'torus.out0.{:05d}.h5'

qualities = ['RHO', 'Pg', 'bsq', 'beta', 'Theta', 'Thetae']

In [3]:
# some constants
cl = 29979245800.0 # speed of light in cgs
mp = 1.67262171e-24
kb = 1.3806505e-16
mu = 1.

In [4]:
def fname(i, src): # closure on `src` and `fmt`
    return (src+fmt).format(i)

In [5]:
src_ma_10M = '/xdisk/chanc/share/GRMHD_dt10M/Ma+0.94_w5/'
src_sa_10M = '/xdisk/chanc/share/GRMHD_dt10M/Sa+0.94_w5/'

src_ma_1000M = '/xdisk/chanc/share/GRMHD_dt1000M/Ma+0.94_w5/'
src_sa_1000M = '/xdisk/chanc/share/GRMHD_dt1000M/Sa+0.94_w5/'

dump0 = pyharm.load_dump(fname(i0, src_ma_10M))
R_low = 1.
R_high = [1, 10, 40, 160]
# first row of SANE model params

In [6]:
def get_dumps(i0, i1, src, step=1):
    dumps = {}
    for i in range(i0, i1, step):
        dumps[str(i)] = pyharm.load_dump(fname(i, src))
    return dumps

In [7]:
def average(irange, dumps, keys):
    s = {k:0 for k in keys}
    for i in irange:
        # print('i:', i)
        # Because data I/O is expensive, we load the data once ...
        d = dumps[str(i)].cache
        # print('d:', d)
        # ... and then sum over multiple keys
        for k in keys:
            s[k] += d[k] # i.e.: s[key1] = file1[key1] + file2[key1] + file3[key1] + ...
            # print('s[k]:', s[k])
            # print('d[k]', d[k])
    print('finish average calculation.')
    return valmap(lambda v: v/len(irange), s) # return an average of variables from the files read in

In [8]:
# works; returns an array of arrays with numerical values
def variance(irange, dumps, quality='Te'):
    """
    quality: str, the physical quality whose variance values we want (e.g. 'Te' for electron temperature)
    dumps: dictionary, all the dump files read in
    """
    key = 'var' + quality
    ss = {key: 0}
    for i in irange:
        d = dumps[str(i)].cache
        ss[key] += (d[quality]-d['avg' + quality])**2
    for i in irange:
        dumps[str(i)].cache[key] = ss[key]
    print('finish variance & standard deviation calculation\'s loop.')
    # return valmap(lambda v: v/len(irange), ss)

$\gamma = \frac{4}{3}$, $x_{e} = \frac{1}{4}$, so ions:electrons = 3:1 (y=1, z=3; $\gamma_{i}=\gamma=\frac{4}{3}, \gamma_e = \gamma + x_{e} / (1 - x_{e})(5/3 - \gamma) = \frac{7}{3}$) <br> 
$Const = m_{p}(\gamma_{e}-1)(\gamma_{i}-1)$ <br> 
$const1 = y(\gamma_{i}-1)$ <br>
$const2 = z(\gamma_{e}-1)$ <br>
$const3 = k_{B}(\gamma-1)$

In [9]:
def get_statistics(dumps, quality):
    """
    dumps: the dumps' dictionary in the format of dumps={'5010':arr1, '5011':arr2, etc}
    quality: string, the quality whose statistics we want 
    """
    print('statistics on ' + quality + ':')
    print('max ' + quality + ':')
    maximum = np.max(dumps[str(i0)].cache[quality])
    print("{:e}".format(maximum))
    print('percentile on ' + quality+ ' [50%, 75%, 90%]:')
    # percentile of the flattened grid
    percentile = np.percentile(dumps[str(i0)].cache[quality], [50, 75, 90])
    print(percentile)

## Get var(Te), var($\rho$), var(Pg), and var($P_{mag}$)

so many calculations...memory issue

In [10]:
def mag_pressure(dump):
    B2 = dump['bsq']
    Pmag = B2/(2*mu)
    
    dump.cache['Pmag'] = Pmag

In [11]:
def R(dump, Rh, Rl, Pmag):
    Pg = dump['Pg']
    return Rl/(Pmag**2+Pg**2)+Rh*Pg**2/(Pmag**2+Pg**2)

In [12]:
# electron temperature
def Te(dump, Rh, Rl, gamma):
    if gamma == 4/3:
        y = 1
        z = 3
        gamma_i = 4/3
        gamma_e = 7/3
    if gamma == 13/9:
        y = 1
        z = 2
        gamma_i = 13/9
        gamma_e = 133/36
    if gamma == 5/3:
        y = 1
        z = 1
        gamma_i = 5/3
        gamma_e = 5/3
    beta = dump['beta']
    u = dump['Pg']/(gamma-1)
    r = R(dump, Rh, Rl)
    numerator = mp*cl**2*u*(gamma_e-1)*(gamma_i-1)
    denom = kb*dump['RHO']*(y*(gamma_i-1)+r*z*(gamma_e-1))
    electron_T = numerator/ denom
    dump.cache['Te'] = electron_T
    return electron_T

In [13]:
src_save_ma = '/home/u12/limengjiang/GRMHD/variances/MAD'
src_save_sa = '/home/u12/limengjiang/GRMHD/variances/SANE'

In [14]:
def save_var(dumps, src_save, Rh):
    for key, dump in dumps.items():
        if Rh == 1.:
            name = fname(key, src_save+'/Rh1')
        if Rh == 160.:
            name = fname(key, src_save+'/Rh160')
        hf = h5py.File(name, 'w')
        d1 = dump.cache['varRHO']
        d2 = dump.cache['varPg']
        d2 = dump.cache['varPmag']
        hf.create_dataset('varRHO', data=d1)
        hf.create_dataset('varPg', data=d2)
        hf.create_dataset('varPmag', data=d2)
        hf.close()

### MAD

In [19]:
# MAD
dumps_ma = get_dumps(i0, i1, src_ma_10M)

In [20]:
dumps_ma[str(i0)].cache.keys()

dict_keys([])

In [17]:
# get ρ
for dump in dumps_ma.values():
    dump['RHO']

In [18]:
# avgRHO
rho_ma_avg1 = average(range(i0, i1), dumps_ma, ['RHO'])
for dump in dumps_ma.values():
    dump.cache['avgRHO'] = np.squeeze(np.array(list(rho_ma_avg1.values())))

KeyError: 'RHO'

In [18]:
# var(ρ)
rho_ma_var1 = variance(range(i0, i1), dumps_ma, 'RHO')
for dump in dumps_ma.values():
    dump.cache['varRHO'] = np.squeeze(np.array(list(rho_ma_var1.values())))

KeyError: 'RHO'

In [18]:
# get Pmag
for dump in dumps_ma.values():
    mag_pressure(dump)

In [20]:
# avgPmag
Pmag_ma_avg1 = average(range(i0, i1), dumps_ma, ['Pmag'])
for dump in dumps_ma.values():
    dump.cache['avgPmag'] = np.squeeze(np.array(list(Pmag_ma_avg1.values())))

finish average calculation.


In [21]:
# var(Pmag)
Pmag_ma_var1 = variance(range(i0, i1), dumps_ma, 'Pmag')
for dump in dumps_ma.values():
    dump.cache['varPmag'] = np.squeeze(np.array(list(Pmag_ma_var1.values())))

finish variance & standard deviation calculation's loop.


AttributeError: 'NoneType' object has no attribute 'values'

In [ ]:
# needed for convolution
get_statistics(dumps_ma, 'varPmag')

In [23]:
# avgPg
Pg_ma_avg1 = variance(range(i0, i1), dumps_ma, 'Pg')
for dump in dumps_ma.values():
    dump.cache['avgPg'] = np.squeeze(np.array(list(Pg_ma_avg1.values())))

KeyError: 'Pg'

In [ ]:
# var(Pg)
Pg_ma_var1 = variance(range(i0, i1), dumps_ma, 'Pg')
for dump in dumps_ma.values():
    dump.cache['varPg'] = np.squeeze(np.array(list(Pg_ma_var1.values())))

$R_{high}=R_{low}=1$

In [ ]:
# get Te
for dump in dumps_ma.values():
    dump.cache['Te'] = Te(dump, R_high[0], Rl, gamma=4/3)

In [ ]:
# var(Te)
Te_ma_var1 = variance(range(i0, i1), dumps_ma)
for dump in dumps_ma.values():
    dump.cache['varTe'] = np.squeeze(np.array(list(Te_ma_var1.values())))

In [ ]:
save_var(dumps_ma, src_save_ma, Rh=1.)

$R_{high}=160$, $R_{low}=1$

In [ ]:
# get Te
for dump in dumps_ma.values():
    dump.cache['Te'] = Te(dump, R_high[3], Rl, gamma=4/3)

In [ ]:
# var(Te)
Te_ma_var2 = variance(range(i0, i1), dumps_ma)
for dump in dumps_ma.values():
    dump.cache['varTe'] = np.squeeze(np.array(list(Te_ma_var2.values())))

In [ ]:
save_var(dumps_ma, src_save_ma, Rh=160.)

### SANE

In [18]:
dumps_sa = get_dumps(i0, i1, src_sa_10M)

In [19]:
dumps_sa[str(i0)].cache.keys()

dict_keys([])

In [ ]:
# get ρ
for dump in dumps_sa.values():
    dump['RHO']

In [ ]:
# avgRHO
rho_sa_avg1 = average(range(i0, i1), dumps_sa, ['RHO'])
for dump in dumps_sa.values():
    dump.cache['avgRHO'] = np.squeeze(np.array(list(rho_sa_avg1.values())))

In [ ]:
# var(ρ)
rho_sa_var1 = variance(range(i0, i1), dumps_sa, 'RHO')
for dump in dumps_sa.values():
    dump.cache['varRHO'] = np.squeeze(np.array(list(rho_sa_var1.values())))

In [ ]:
# get Pmag
for dump in dumps_sa.values():
    mag_pressure(dump)

In [ ]:
# avgPmag
Pmag_sa_avg1 = average(range(i0, i1), dumps_sa, ['Pmag'])
for dump in dumps_sa.values():
    dump.cache['avgPmag'] = np.squeeze(np.array(list(Pmag_sa_avg1.values())))

In [ ]:
# var(Pmag)
Pmag_sa_var1 = variance(range(i0, i1), dumps_sa, 'Pmag')
for dump in dumps_sa.values():
    dump.cache['varPmag'] = np.squeeze(np.array(list(Pmag_sa_var1.values())))

In [ ]:
get_statistics(dumps_sa, 'varPmag')

In [ ]:
# avgPg
Pg_sa_avg1 = variance(range(i0, i1), dumps_sa, 'Pg')
for dump in dumps_sa.values():
    dump.cache['avgPg'] = np.squeeze(np.array(list(Pg_sa_avg1.values())))

In [ ]:
# var(Pg)
Pg_sa_var1 = variance(range(i0, i1), dumps_sa, 'Pg')
for dump in dumps_sa.values():
    dump.cache['varPg'] = np.squeeze(np.array(list(Pg_sa_var1.values())))

$R_{high}=R_{low}=1$

In [ ]:
# get Te
for dump in dumps_sa.values():
    dump.cache['Te'] = Te(dump, R_high[0], Rl, gamma=4/3)

In [ ]:
# var(Te)
Te_sa_var1 = variance(range(i0, i1), dumps_sa)
for dump in dumps_sa.values():
    dump.cache['varTe'] = np.squeeze(np.array(list(Te_sa_var1.values())))

In [ ]:
save_var(dumps_sa, src_save_sa, Rh=1.)

$R_{high}=160$, $R_{low}=1$

In [ ]:
# get Te
for dump in dumps_sa.values():
    dump.cache['Te'] = Te(dump, R_high[3], Rl, gamma=4/3)

In [ ]:
# var(Te)
Te_sa_var2 = variance(range(i0, i1), dumps_sa)
for dump in dumps_sa.values():
    dump.cache['varTe'] = np.squeeze(np.array(list(Te_sa_var2.values())))

In [ ]:
save_var(dumps_ma, src_save_ma, Rh=160.)